In [1]:
from tkinter import *
from tkinter import ttk
import time
import random
import difflib

class MainWindow:
    def __init__(self, root):
        self.texts = [
            "In the heart of a bustling metropolis, where the symphony of city life plays out in a cacophony of honking taxis, murmuring crowds, and the rhythmic hum of subway trains, there lies a tranquil garden nestled among the steel and glass giants. Amidst the lush greenery and the delicate fragrance of blooming flowers, one can find a momentary escape from the frenetic pace of urban existence. The garden, with its winding cobblestone paths and serene reflecting pools, serves as a sanctuary where the gentle rustling of leaves and the occasional chirp of a bird offer a soothing counterpoint to the city's incessant din. Here, time seems to slow down, allowing visitors to savor the simple pleasures of nature and find solace in the stillness that contrasts sharply with the surrounding hustle and bustle. As the sun casts long shadows across the garden's meticulously manicured lawns and the evening light begins to fade, the space transforms into a canvas of tranquil hues, inviting quiet reflection and a sense of peace that is all too rare in the clamor of modern life."
        ]
        self.speed = 0
        self.accuracy = 0
        self.time_start = 0
        self.time_end = 0
        self.timer_running = False

        root.title("Speed Typing Test")
        root.minsize(600, 400)
        root.grid_rowconfigure(0, weight=1)
        root.grid_rowconfigure(1, weight=1)
        root.grid_rowconfigure(2, weight=1)
        root.grid_rowconfigure(3, weight=1)
        root.grid_rowconfigure(4, weight=1)
        root.grid_rowconfigure(5, weight=1)
        root.grid_columnconfigure(0, weight=1)
        root.grid_columnconfigure(1, weight=1)
        root.grid_columnconfigure(2, weight=1)

        self.label_text = Label(root, text="Welcome to Speed Typing Test.", wraplength=550)
        self.label_text.grid(row=0, column=0, columnspan=3, sticky="nsew", pady=10)

        self.user_text = Text(root, wrap=WORD, height=8)
        self.user_text.grid(row=1, column=0, columnspan=3, sticky="nsew", padx=10, pady=10)
        self.user_text.bind('<KeyRelease>', self.on_key_release)

        self.btn_start = Button(root, text="Start/Restart", command=self.start)
        self.btn_start.grid(row=2, column=0, sticky="nsew", padx=10, pady=10)
        self.btn_stop = Button(root, text="Stop", command=self.stop)
        self.btn_stop.grid(row=2, column=1, sticky="nsew", padx=10, pady=10)
        self.btn_newtext = Button(root, text="New Text", command=self.new_text)
        self.btn_newtext.grid(row=2, column=2, sticky="nsew", padx=10, pady=10)

        self.label_speed = Label(root, text=f"Your typing speed is {self.speed} WPM")
        self.label_speed.grid(row=3, column=0, columnspan=3, sticky="nsew", pady=5)

        self.label_accuracy = Label(root, text=f"Your typing accuracy is {self.accuracy} %")
        self.label_accuracy.grid(row=4, column=0, columnspan=3, sticky="nsew", pady=5)

        self.label_timer = Label(root, text="Time: 0 s")
        self.label_timer.grid(row=5, column=0, columnspan=3, sticky="nsew", pady=5)

    def start(self):
        self.time_start = time.time()
        self.timer_running = True
        self.update_timer()  # Start the timer updates
        self.user_text.delete('1.0', END)
        self.user_text.focus_set()

    def stop(self):
        if not self.timer_running:
            return
        
        self.time_end = time.time()
        self.timer_running = False
        
        words = self.label_text.cget("text").split()
        typed_words = self.user_text.get("1.0", 'end-1c').split()
        self.speed = round(len(typed_words) / ((self.time_end - self.time_start) / 60))
        
        self.label_speed.config(text=f"Your typing speed is {self.speed} WPM")
        self.accuracy = round(difflib.SequenceMatcher(None, self.label_text.cget("text"), ' '.join(typed_words)).ratio() * 100)
        self.label_accuracy.config(text=f"Your typing accuracy is {self.accuracy} %")

    def new_text(self):
        self.label_text.config(text=self.texts[random.randint(0, len(self.texts) - 1)])
        self.user_text.delete('1.0', END)

    def on_key_release(self, event):
        if self.timer_running:
            input_text = self.user_text.get("1.0", 'end-1c')
            reference_text = self.label_text.cget("text")
            self.highlight_text(input_text, reference_text)

    def highlight_text(self, input_text, reference_text):
        self.user_text.tag_remove('highlight', '1.0', 'end')
        correct_text = reference_text.split()
        typed_text = input_text.split()
        
        index = '1.0'
        for i, word in enumerate(correct_text):
            end_index = f"{index}+{len(word)}c"
            if i < len(typed_text):
                if word == typed_text[i]:
                    self.user_text.tag_add('highlight', index, end_index)
                    self.user_text.tag_configure('highlight', background='lightgreen')
                else:
                    self.user_text.tag_add('highlight', index, end_index)
                    self.user_text.tag_configure('highlight', background='red')
            index = end_index

    def update_timer(self):
        if self.timer_running:
            elapsed_time = int(time.time() - self.time_start)
            self.label_timer.config(text=f"Time: {elapsed_time} s")
            root.after(1000, self.update_timer)  # Update every second

def main():
    global root
    root = Tk()
    app = MainWindow(root)
    root.mainloop()

if __name__ == "__main__":
    main()

